In [1]:
import pandas as pd
import csv

path_base="/Users/leepand/Downloads/github/rec-a-sketch-master"
df_likes = pd.read_csv(path_base+'/data/model_likes_anon.psv',
                 sep='|', quoting=csv.QUOTE_MINIMAL,
                 quotechar='\\')
df_likes.drop_duplicates(inplace=True)
df_likes.head()


sideinfo = pd.read_csv(path_base+'/data/model_feats.psv',
                       sep='|', quoting=csv.QUOTE_MINIMAL,
                       quotechar='\\')

In [3]:
sideinfo.head()

,mid,type,value
0,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,category,Characters
1,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,category,Gaming
2,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,tag,3dsmax
3,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,tag,noel
4,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,tag,loli


In [2]:
#from lightfm.data import Dataset
from bolt4ds.recsys.data import Dataset

data = Dataset()
data.fit(df_likes.uid.unique(), df_likes.mid.unique())
num_users, num_items = data.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))


interactions_matrix,weights_matrix = data.build_interactions([(i[1],i[0])for i in df_likes.drop(['modelname'], axis = 1).values])

item_feats = data.buid_sparse_item_features(sideinfo)

Num users: 62583, num_items 28806.
(28806, 21951)


In [9]:
from web.py3helpers import (
    is_iter,
    iteritems,
    itervalues,
)

def dictreverse(mapping):
    """
    Returns a new dictionary with keys and values swapped.
        >>> dictreverse({1: 2, 3: 4})
        {2: 1, 4: 3}
    """
    return dict([(value, key) for (key, value) in iteritems(mapping)])

items_mapping  = data.mapping()[2]
item_ids = dictreverse(items_mapping)
idx_to_mid = item_ids

In [10]:
import bolt4ds.sparsity as sp3
import numpy as np
from bolt4ds.recsys.algo.lightfm_auto import LFMRecommender

test=LFMRecommender(indicators="both",loss='warp', random_state=2020)

test.fit_partial(sp3.SparseFrame(interactions_matrix))
items = df_likes.mid.index.unique()
#df_likes.mid.unique()
idx_to_mid = item_ids
scores = test.predict_online(user_id=625844, item_ids= list(idx_to_mid.keys()), item_features=None,
                             user_features=None, num_threads=3, use_baseline=False)

#i_idx = [x for x in np.argsort(-scores)]
scores

0        2.158481
1        2.185728
2        1.983770
3        1.921216
4        1.689276
           ...   
28801   -1.265289
28802   -1.328992
28803   -0.748338
28804   -1.194334
28805   -0.537891
Length: 28806, dtype: float64

In [11]:
test2=LFMRecommender(indicators="both",loss='warp', random_state=2020)
item_features= sp3.SparseFrame(item_feats)
test2.fit_partial(sp3.SparseFrame(interactions_matrix),item_features=item_features)


In [12]:
scores2 = test2.predict_online(user_id=625844, item_ids= list(idx_to_mid.keys()), item_features=item_features,
                             user_features=None, num_threads=3, use_baseline=False)

scores2

0        6.532302
1        5.458549
2        5.380723
3        4.790645
4        4.219454
           ...   
28801   -0.176589
28802   -2.519736
28803   -0.221245
28804   -2.177250
28805   -0.954533
Length: 28806, dtype: float64